WIP
Citations: Andrew Lucas: Ising formulations of many NP Problems. arXiv:1302.5843. 
https://arxiv.org/abs/1302.5843

Ising machine: Number Partitioning

Number partitioning is an algorithm which partitions a set of numbers into two
sets such that both sets add up to the same amount. 

In [2]:
import numpy as np
import scienceplots
import matplotlib.pyplot as plt
import numba
import math


In [4]:
# generate a random set of N positive whole numbers
def generate_data(N):
    return np.random.randint(1, N, N) # low, high, size

N = 100
data = generate_data(N)
print("data: ", data)
print("length:", len(data))
print("Data generated.")

data:  [19 95 91 85 44 55  5 38 23 66 53  2 23 45 60 66 85 44 87 80 25 10 17 26
 23 35 94 15 12 43 55 78  6 55  6 29 45 56 30 75 38 12 59 51 54 35 84 69
 60 82 48 34 34 94 94 57 15 65 70 80 61 18 41 15 58 79 98 22 18 38 33 49
 43 34 46 34 34 25 61 38 85 53 82 68 50 35 51 72  4 62 23 52 45 44 71 34
 75 59 27 51]
length: 100
Data generated.


In [5]:
# generate an initial N x 1 vector of spins which are -1 or 1
def generate_spins(N):
    return np.random.choice([-1, 1], N) # 1D array, size. generates random sample

vector = generate_spins(N)
print(vector)

[-1 -1 -1 -1  1  1 -1  1  1 -1  1  1 -1  1 -1 -1  1 -1 -1  1 -1  1 -1  1
 -1 -1 -1 -1 -1 -1 -1 -1  1  1  1  1 -1 -1  1 -1  1 -1 -1 -1  1 -1 -1  1
  1 -1  1  1  1 -1  1 -1 -1  1  1 -1 -1  1  1  1  1  1 -1  1  1 -1  1  1
  1  1 -1  1 -1 -1 -1  1 -1 -1 -1  1 -1  1 -1  1 -1 -1 -1 -1 -1  1  1 -1
 -1 -1  1 -1]


In [8]:
# partitions numbers randomly into two groups based on the spin vector
def get_groups(numbers_set, spin_vector):
    N = len(spin_vector)
    
    set_A = np.empty(0) # []
    set_B = np.empty(0)

    for n in range(N):
        if spin_vector[n] == 1:
            set_A = np.append(set_A, numbers_set[n])
        elif spin_vector[n] == -1: 
            set_B = np.append(set_B, numbers_set[n])
    return set_A, set_B

def get_sums(set_A, set_B):
    return sum(set_A), sum(set_B)

set_A, set_B = get_groups(data, vector) # data, vector generated previously

print("set A: ", set_A)
print("set_B", set_B)

sum_A, sum_B = get_sums(set_A, set_B)
print("sum A: ", sum_A)
print("sum B: ", sum_B)

set A:  [44. 55. 38. 23. 53.  2. 45. 85. 80. 10. 26.  6. 55.  6. 29. 30. 38. 54.
 69. 60. 48. 34. 34. 94. 65. 70. 18. 41. 15. 58. 79. 22. 18. 33. 49. 43.
 34. 34. 38. 68. 35. 72. 44. 71. 27.]
set_B [19. 95. 91. 85.  5. 66. 23. 60. 66. 44. 87. 25. 17. 23. 35. 94. 15. 12.
 43. 55. 78. 45. 56. 75. 12. 59. 51. 35. 84. 82. 94. 57. 15. 80. 61. 98.
 38. 46. 34. 25. 61. 85. 53. 82. 50. 51.  4. 62. 23. 52. 45. 34. 75. 59.
 51.]
sum A:  1952.0
sum B:  2872.0


In [10]:
def outer_prod(data):
    S = np.array(data)
    N = len(S)
    J = np.outer(S, S)
    
    return J

J = outer_prod(data)
print("J:", J)
# A is a constant for the Hamiltonian
A = 1

# H(s) = s^T * J * s
def compute_init_energy(J, s):
    return s @ J @ s # dimensions are handled automatically.


def compute_delta_energy(a, s, i):
    summation = np.sum(a[i] * a * s) # sum across all j 
    return - 2 * s[i] * summation

J: [[ 361 1805 1729 ... 1121  513  969]
 [1805 9025 8645 ... 5605 2565 4845]
 [1729 8645 8281 ... 5369 2457 4641]
 ...
 [1121 5605 5369 ... 3481 1593 3009]
 [ 513 2565 2457 ... 1593  729 1377]
 [ 969 4845 4641 ... 3009 1377 2601]]


In [18]:
def metropolis(a_vector, spin_vector, temp, num_steps):
    N = len(a_vector)
    J = outer_prod(a_vector)
    E_i = compute_init_energy(J, spin_vector)
    current_energy = E_i.copy()
    lowest_energy = current_energy.copy()
    best_sort = spin_vector.copy()
    print("Initial energy:", current_energy)

    for step in range(num_steps):
        # choose a random number to switch into the other group
        i = np.random.randint(0, N)
        # compute the change in energy that would result from flipping to the
        # other group
        delta_E = compute_delta_energy(a_vector, spin_vector, i)
        # print delta_E for first couple of steps only
        if step < 10:
            print(f"Step {step}: delta_E = {delta_E}")

        # if the energy change is negative, accept the change or accept with a probability.
        if delta_E <= 0 or np.random.rand() < np.exp(-delta_E / temp):
            spin_vector[i] *= -1  # Flip the spin officially moving that number to the other set
            current_energy += delta_E  # Update the current energy
            # If the current energy is lower than the best found so far
            if current_energy < lowest_energy:
                lowest_energy = current_energy.copy()
                best_sort = spin_vector.copy()

    return best_sort, lowest_energy, current_energy, spin_vector


temp = 1.0
num_steps = 20000
spin_vector = generate_spins(N)
a_vector = generate_data(N)

# Initial Groups and Sums
set_A, set_B = get_groups(a_vector, spin_vector)
sum_A, sum_B = get_sums(set_A, set_B)
print("Initial set A:", set_A)
print("Initial set B:", set_B)

print("Initial sum A:", sum_A)
print("Initial sum B:", sum_B)


best_sort, lowest_energy, current_energy, spin_vec = metropolis(a_vector, spin_vector, temp, num_steps)

print("Best sort:", best_sort)
print("Lowest energy:", lowest_energy)
print("Current energy:", current_energy)
print("Final spin vector:", spin_vec)

# find the groups and sums
set_A, set_B = get_groups(a_vector, spin_vec)
sum_A, sum_B = get_sums(set_A, set_B)
print("Final set A:", set_A)
print("Final set B:", set_B)

print("Final sum A:", sum_A)
print("Final sum B:", sum_B)



Initial set A: [98.  4. 16. 93. 84. 27. 47. 69. 36. 63. 20.  7. 25. 89. 51. 87. 56. 61.
 11.  8. 36.  9.  2.  6. 59.  3. 34. 42. 19. 12. 15. 56.  3. 73. 16. 17.
 95. 79. 38. 64. 14.  6. 59. 83. 38. 88. 81. 58. 75. 46. 89. 69. 14. 74.
 76.]
Initial set B: [ 5. 15. 78.  2. 11.  9. 91.  9. 91. 22. 76. 73. 49. 75. 96. 31. 52. 63.
 40. 24. 19. 67. 50. 73. 94. 14. 45. 73.  5.  2. 28.  4. 10. 66. 70.  9.
 19. 95. 98. 83. 33. 91. 24. 10. 58.]
Initial sum A: 2500.0
Initial sum B: 2052.0
Initial energy: 200704
Step 0: delta_E = -65408
Step 1: delta_E = 42280
Step 2: delta_E = -35032
Step 3: delta_E = 23436
Step 4: delta_E = 3348
Step 5: delta_E = -13392
Step 6: delta_E = 10260
Step 7: delta_E = 16644
Step 8: delta_E = 2052
Step 9: delta_E = 16644
Best sort: [-1  1 -1 -1 -1  1  1 -1  1  1 -1  1  1  1 -1 -1 -1 -1  1 -1  1  1  1  1
  1  1  1  1  1 -1  1  1 -1  1  1 -1  1 -1  1 -1 -1  1 -1  1  1 -1  1 -1
 -1 -1 -1  1  1  1 -1 -1 -1 -1  1 -1 -1 -1 -1  1 -1  1  1  1  1 -1  1  1
 -1  1 -1  1  1 -1 -1 -